## 初始化 SecretFlow

alice 和 bob 节点都需要初始化 secretflow。首先在两个节点分别选取一个可以被对方访问的地址，注意，端口号要选取未被占用的端口。


In [ ]:
import secretflow as sf
import spu
import os

network_conf = {
    "parties": {
        "alice": {
            "address": "alice:8000",
        },
        "bob": {
            "address": "bob:8000",
        },
    },
}

party = os.getenv("SELF_PARTY", "alice")
sf.shutdown()
sf.init(
    address="127.0.0.1:6379",
    cluster_config={**network_conf, "self_party": party},
    log_to_driver=True,
)


## 初始化 SPU

1. alice 的 address 请填写可以被 bob 访通的地址，并且选择一个**未被占用的端口** ，注意不要和 Ray 端口冲突。
2. alice 的 listen_addr 可以和 alice address 里的端口一样。
3. bob 的 address 请填写可以被 alice 访通的地址，并且选择一个**未被占用的端口** ，注意不要和 Ray 端口冲突。
4. bob 的 listen_addr 可以和 bob address 里的端口一样。


In [ ]:
alice, bob = sf.PYU("alice"), sf.PYU("bob")
spu_conf = {
    "nodes": [
        {
            "party": "alice",
            "address": "alice:8001",
            "listen_addr": "alice:8001",
        },
        {
            "party": "bob",
            "address": "bob:8001",
            "listen_addr": "bob:8001",
        },
    ],
    "runtime_config": {
        "protocol": spu.spu_pb2.SEMI2K,
        "field": spu.spu_pb2.FM128,
        "sigmoid_mode": spu.spu_pb2.RuntimeConfig.SIGMOID_REAL,
    },
}
spu = sf.SPU(cluster_def=spu_conf)


## 隐私求交

<span style="color: rgb(50, 50, 50)">提供 </span>`psi_csv`<span style="color: rgb(50, 50, 50)"> 函数， </span>`psi_csv`<span style="color: rgb(50, 50, 50)"> 将 csv 文件作为输入，并在求交后生成 csv 文件。默认协议为 </span>[**KKRT**](https://eprint.iacr.org/2016/799.pdf)<span style="color: rgb(50, 50, 50)">。</span>


In [ ]:
current_dir = os.getcwd()

input_path = {
    alice: f"{current_dir}/iris_alice.csv",
    bob: f"{current_dir}/iris_bob.csv",
}
output_path = {
    alice: f"{current_dir}/iris_alice_psi.csv",
    bob: f"{current_dir}/iris_alice_psi.csv",
}
spu.psi_csv("uid", input_path, output_path, "alice")
